### Created a portfolio using BAB strategy
- BAB strategy - The basic bet against beta strategy is to find assets with higher bets and take a short position in them. 
- The idea is the higher beta assets are overpriced and the lower beta assets are underpriced.
- *Note:* BAB strategy is to buy high-beta stocks and sell low-beta stocks

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import glob
import yfinance as yf

In [ ]:
os.getcwd()

In [ ]:
# read nifty50list company
nifty_list = pd.read_csv('ind_nifty50list.csv')
nifty_list.head()

In [ ]:
#extracting the symbol of the nifty50 list company
nifty_symbol=nifty_list['Symbol']
nifty_symbol

In [ ]:
#create an empty list to store the symbol of the downloaded and notdownloaded stock
downloaded_list = []
not_downloaded_list = []

In [ ]:
os.getcwd()

In [ ]:
#change the working directory to stock the csv file in seperate directory
os.chdir('D:\\stock analysis\\portfolio\\nifty_stock_price')
os.getcwd()

In [ ]:
nifty_df = yf.download('^NSEI','2021-01-27','2021-01-28',interval='1m')

In [ ]:
nifty_df
nifty_df['Name'] = "^NSEI"
nifty_df.to_csv("^NSEI"+'.csv')

In [ ]:
#download 1minute date and store in csv file
for symbol in nifty_list['Symbol']:
    try:
        print(symbol)
        yahoo_symbol = symbol+'.NS'
        stock_df = yf.download(yahoo_symbol,'2021-01-27','2021-01-28',interval='1m')
        stock_df['Name'] = yahoo_symbol
        downloaded_list.append(yahoo_symbol)
        stock_df.to_csv(yahoo_symbol+'.csv')
        print('successfully downloaded...%s\n'%(yahoo_symbol))
    except:
        not_downloaded_list = []
        print('Problem in downloading the %s\n' %(symbol))

### combine all close price with col_name as cpy name

In [ ]:
filename = glob.glob('D:/stock analysis/portfolio/nifty_stock_price/*.csv')
filename

In [ ]:
df=((pd.read_csv(file,index_col='Datetime',parse_dates=True)[['Close']])
          .rename(columns={'Close':symbol}) 
          for (file,symbol) in zip(filename,nifty_symbol))

In [ ]:
final_df=pd.concat(df,axis=1)

In [ ]:
final_df

In [ ]:
final_df['^NSEI'] = nifty_df['Close']

In [ ]:
final_df

In [ ]:
final_df.reset_index(inplace=True)

In [ ]:
final_df

In [ ]:
final_df['Datetime'] = pd.to_datetime(final_df['Datetime']).dt.strftime('%H:%M')

In [ ]:
final_df.set_index('Datetime',inplace=True)

In [ ]:
final_df

### daily percentage change

In [ ]:
daily_pct_change = final_df.pct_change().dropna()

In [ ]:
daily_pct_change

### calculate the beta

In [ ]:
import statsmodels.api as sm

In [ ]:
def calc_beta(y,x):
    model = sm.OLS(y,x)
    results = model.fit()
    return results.params[0]

In [ ]:
beta = pd.DataFrame(index=[0])
beta


In [ ]:
for ticker in daily_pct_change.columns:
    beta[ticker] = calc_beta(daily_pct_change.loc[:,ticker],daily_pct_change.loc[:,'^NSEI'])
beta = beta.T

In [ ]:
beta

In [ ]:
beta.plot.bar(figsize=(18,7))
plt.show()

### Alpha 1:Beta<0.7 

In [ ]:
beta.plot.bar(figsize=(18,7),color=np.where(beta<1,'g','r').T)
plt.axhline(1, linestyle='-.',color='blue')
plt.show()

In [ ]:
low_beta = beta[beta.values<1].index
low_beta

In [ ]:
def plot_performance(stock_list,strategy_name):
    stock_returns = daily_pct_change.loc[:,stock_list]
    (stock_returns+1).cumprod().plot(figsize=(10,7),legend="left")
    plt.title(strategy_name)
    plt.show()
    
    portfolio = stock_returns.mean(axis=1)
    (portfolio+1).cumprod().plot(figsize=(10,7),label='BAB + ROE')
    plt.title(strategy_name + ' portfolio performance')
    
    nifty = daily_pct_change.loc[:,'^NSEI']
    (nifty+1).cumprod().plot(figsize=(10,7),label='Nifty')
    plt.legend()
    plt.show

In [ ]:
plot_performance(low_beta,'Low Beta')

In [ ]:
high_beta = beta[beta.values >= 1].index
high_beta

In [ ]:
plot_performance(high_beta,'High Beta')